<img src="https://openfisca.org/img/logo-openfisca.svg" width="198" align="right">
<h1 id="tocheading">Getting Started</h1>

This notebook illustrates the [key concepts](https://openfisca.org/doc/key-concepts/index.html) of OpenFisca that you will find in the documentation.

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>


## Calculate a variable

Do some imports

In [ ]:
from openfisca_france import FranceTaxBenefitSystem

Initialize the french [Tax and Benefit system](https://openfisca.org/doc/key-concepts/tax_and_benefit_system.html) : it launches the entire legislation.

In [ ]:
tax_benefit_system = FranceTaxBenefitSystem()

Create a [scenario](https://openfisca.org/doc/key-concepts/input_data.html#scenario): first step to give a familial situation for which you want to calculate taxes or benefits.

In [ ]:
scenario = tax_benefit_system.new_scenario()

Initialize a test case (the familial situation) in the scenario:
here we have one parent and two children.

In [ ]:
scenario.init_single_entity(
    period = 2015,
    parent1 = dict(
        age = 30,
        salaire_de_base = 20000,
        ),
    enfants = [
        dict(age = 12),
        dict(age = 18),
        ],
    )

Create a [simulation](https://openfisca.org/doc/key-concepts/simulation.html#simulation-the-framework-of-computation): the OpenFisca frame for calculating taxes or benefits.

In [ ]:
simulation = scenario.new_simulation()

Calculate a variable : `"af"` (allocations familiales) for example.

In [ ]:
simulation.calculate('af', '2015-01')

array([129.99], dtype=float32)

## Test the impact of a reform

A reform represents a modified version of the real Tax and Benefit legislation* 

Reforms are gather in the module [`Reform`](https://github.com/openfisca/openfisca-france/tree/master/openfisca_france/reforms) of OpenFisca.

You have then to import the module extension corresponding to the reform of your concern.

Here we choose the fiscal reform of Landais, Piketty and Saez on the income tax (http://www.revolution-fiscale.fr/la-reforme-proposee). 

It is already implemented in OpenFisca under the name `landais_piketty_saez` .

In [ ]:
from openfisca_france.reforms import landais_piketty_saez

Create a modified version of the tax and benefit system, affected by the changes introduced by the reform

In [ ]:
reform = landais_piketty_saez.landais_piketty_saez(tax_benefit_system)

Create the scenario you want to test.

In [ ]:
def init_profile(scenario):
    scenario.init_single_entity(
        period = '2013',
        parent1 = dict(
            age = 40,
            salaire_de_base = 50000,
            ),
        )
    return scenario

With the reform

In [ ]:
#Indicate that you want to perfom the reform on this scenario
reform_scenario = init_profile(reform.new_scenario())

#Simulate the reform
reform_simulation = reform_scenario.new_simulation() 

# Choose the variable you want to calcul : here the disposable income, "revenu_disponible"
reform_simulation.calculate('revenu_disponible', '2013')

array([13554.031], dtype=float32)

Without the reform: the counterfactual

In [ ]:
#Indicate that you want to perfom the standard system on this scenario
reference_scenario = init_profile(tax_benefit_system.new_scenario())

#Simulate the standard scenario
reference_simulation = reference_scenario.new_simulation()

# Choose the variable you want to calcul
reference_simulation.calculate('revenu_disponible', '2013')

array([33978.87], dtype=float32)

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>